In [15]:
import json
import os
import numpy as np
import cv2 as cv
from skimage.color import deltaE_ciede2000


In [16]:
with open('../sources/metadata/image_database.json', 'r') as file:
    image_database = json.load(file)
    
all_images = (
    image_database[0]['images']
    + image_database[1]['images']
    + image_database[2]['images']
    + image_database[3]['images']
)

In [17]:
colors_output = {}

In [18]:
# load color categories from all images
def similar_image_by_color(color, data, similarity_threshold = 50):
    similar_images = []
    for element in data:
        try:  
            image_path = element['path']
            palette = element['color_palette']
            score = 0
            for sub_color in palette:
                sub = np.zeros((1, 1, 3), dtype=np.uint8)
                sub[0, 0] = [sub_color[0], sub_color[1], sub_color[2]]
                lab = cv.cvtColor(sub, cv.COLOR_RGB2LAB)
                sub_color = lab[0][0]
                diff = deltaE_ciede2000(color, sub_color)
                score += diff
            score /= len(palette)
            if score < similarity_threshold:
                similar_images.append({
                    'path': image_path,
                    'score': score
                })
        except Exception as e:
            print(f"Error extracting color palette from {image_path} : {str(e)}")
    return sorted(similar_images, key=lambda item: item['score'], reverse=False)

colors_names = [
    'white', 'black', 'red', 'green', 'blue',
    'orange', 'yellow', 'purple', 'pink', 'brown',
    'gray', 'maroon', 'turquoise', 'cyan', 'navy blue',
    'gold', 'silver', 'beige', 'teal', 'lime',
    'olive', 'violet', 'magenta', 'indigo'
]

colors = [
    [255, 255, 255],
    [0, 0, 0],
    [255, 0, 0],
    [0, 255, 0],
    [0, 0, 255],
    [255, 165, 0],
    [255, 255, 0],
    [128, 0, 128],
    [255, 192, 203],
    [165, 42, 42],
    [128, 128, 128],
    [128, 0, 0],
    [64, 224, 208],
    [0, 255, 255],
    [0, 0, 128],
    [255, 215, 0],
    [192, 192, 192],
    [245, 245, 220],
    [0, 128, 128],
    [0, 255, 0],
    [128, 128, 0],
    [238, 130, 238],
    [255, 0, 255],
    [75, 0, 130]
]
for color, color_name in zip(colors, colors_names):
    image = np.zeros((1, 1, 3), dtype=np.uint8)
    image[0, 0] = color
    lab = cv.cvtColor(image, cv.COLOR_RGB2LAB)
    color = lab[0][0]

    similar = similar_image_by_color(color, all_images, similarity_threshold=50)
    if color_name not in colors_output:
        colors_output[color_name] = [element['path'] for element in similar]

Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\camera\000054.jpg : division by zero


C:\Users\DELL\AppData\Local\Temp\ipykernel_16904\2164914489.py:11: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 256 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  sub[0, 0] = [sub_color[0], sub_color[1], sub_color[2]]


Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\camera\000092.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\summer\000218.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\camera\000054.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\camera\000092.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\summer\000218.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\camera\000054.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\camera\000092.jpg : division by zero
Error extracting color palette from D:\image_search_engine_ai-end\sources\photos\summer\000218.jpg : division by zero
Error extracting color palette from D:\image_search_engi

In [19]:
# load background classes from all images
with open('../sources/metadata/image_background_pair.json', 'r') as file:
    background  = json.load(file)

background_output = {}
for class_name in background:
    image_paths = [all_images[i]["path"] for i in background[class_name]]
    if class_name not in background_output:
        background_output[class_name] = image_paths

In [20]:
# load faces sets from camera folder
faces_output = {}
for element in image_database[0]["images"]:
    try:
        for label in element["persons"]:
            if label not in faces_output:
                faces_output[label] = []
            faces_output[label].append(element["path"])
    except Exception as e:
        print(f"Error extracting face from {element['path']} : {str(e)}")

Error extracting face from D:\image_search_engine_ai-end\sources\photos\camera\000054.jpg : 'persons'
Error extracting face from D:\image_search_engine_ai-end\sources\photos\camera\000092.jpg : 'persons'


In [21]:
# load objects classes from all images
objects_output = {}
for element in all_images:
    try:
        for label in list(element["objects"]["categories"].keys()):
            if label not in objects_output:
                objects_output[label] = []
            objects_output[label].append(element["path"])
    except Exception as e:
        print(f"Error extracting face from {element['path']} : {str(e)}")

In [24]:
output = {
    "colors_keys": list(colors_output.keys()),
    "colors" : colors_output,
    "faces_keys": list(faces_output.keys()),
    "faces" : faces_output,
    "backgrounds_keys": list(background_output.keys()),
    "backgrounds" : background_output,
    "objects_keys": list(objects_output.keys()),
    "objects": objects_output
}

In [25]:
with open('../sources/metadata/home_page_filters_option.json', 'w') as f:
    json.dump(output, f, indent=4)